<a href="https://colab.research.google.com/github/VenGr0/LLM_with_KnowledgeGraph/blob/main/KnowledgeGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подготовка данных для выполнения ДЗ

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install llama_index pyvis Ipython langchain pypdf langchain_community
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-langchain
!pip install langchain-huggingface
!pip install sentencepiece accelerate
!pip install -U bitsandbytes
!pip install peft

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-m6y1ndqg
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-m6y1ndqg
  Resolved https://github.com/huggingface/transformers to commit 80b774eb2906c34265c54b9534758ffc0d619cb7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9763989 sha256=55795614cefcf23d6441be00e061e840aa6b633b10ba2d491b70631595f0ad22
  Stored in directory: /tmp/pip-ephem-wheel-cache-wiuh7q1q/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
  Using cached jedi-0.19.1-py2.py3-none-

In [ ]:
!pip install llama-index-readers-file

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import KnowledgeGraphIndex
from llama_index.core import Settings
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

#from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from pyvis.network import Network

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPI has co

In [ ]:
from huggingface_hub import login
HF_TOKEN="#подставляем свой токен HF"
# Вставьте ваш токен (здесь указан временный токен)
login(HF_TOKEN, add_to_git_credential=True)


Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        elif message.role == 'bot':
            prompt += f"<s>bot\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<s>system\n"):
        prompt = "<s>system\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<s>bot\n"
    return prompt

def completion_to_prompt(completion):
    return f"<s>system\n</s>\n<s>user\n{completion}</s>\n<s>bot\n"

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

# Определяем параметры квантования, иначе модель не выполниться в колабе
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Задаем имя модели
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"

# Создание конфига, соответствующего методу PEFT (в нашем случае LoRA)
config = PeftConfig.from_pretrained(MODEL_NAME)

# Загружаем базовую модель, ее имя берем из конфига для LoRA
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,          # идентификатор модели
    quantization_config=quantization_config, # параметры квантования
    torch_dtype=torch.float16,               # тип данных
    device_map="auto"                        # автоматический выбор типа устройства
)

# Загружаем LoRA модель
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)

# Переводим модель в режим инференса
# Можно не переводить, но явное всегда лучше неявного
model.eval()

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [ ]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [ ]:
llm = HuggingFaceLLM(
    model=model,             # модель
    model_name=MODEL_NAME,   # идентификатор модели
    tokenizer=tokenizer,     # токенизатор
    max_new_tokens=generation_config.max_new_tokens, # параметр необходимо использовать здесь, и не использовать в generate_kwargs, иначе ошибка двойного использования
    model_kwargs={"quantization_config": quantization_config}, # параметры квантования
    generate_kwargs = {   # параметры для инференса
      "bos_token_id": generation_config.bos_token_id, # токен начала последовательности
      "eos_token_id": generation_config.eos_token_id, # токен окончания последовательности
      "pad_token_id": generation_config.pad_token_id, # токен пакетной обработки (указывает, что последовательность ещё не завершена)
      "no_repeat_ngram_size": generation_config.no_repeat_ngram_size,
      "repetition_penalty": generation_config.repetition_penalty,
      "temperature": generation_config.temperature,
      "do_sample": True,
      "top_k": 50,
      "top_p": 0.95
    },
    messages_to_prompt=messages_to_prompt,     # функция для преобразования сообщений к внутреннему формату
    completion_to_prompt=completion_to_prompt, # функции для генерации текста
    device_map="auto",                         # автоматически определять устройство
)

In [ ]:
inputs = [
    "Вопрос: Почему трава зеленая?",
    "Задание: Сочини короткий рассказ, обязательно упоминая следующие объекты: Таня, мячик, речка",
    "Рецепт борща."
]

In [ ]:
for num, text in enumerate(inputs):
  print(inputs[num])
  response = llm.complete(inputs[num])
  print(str(response))
  print('='*50)

Вопрос: Почему трава зеленая?


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Трава зеленая из-за того, что она содержит хлорофилл. Хлорофилл - это пигмент, который позволяет растениям получать энергию от солнечного света и превращать его в углеводы (карбонаты) путем фотосинтеза. Фотосинтез - это процесс, благодаря которому растения могут производить свою собственную пищу, используя свет, воду и углекислый газ из окружающей среды.
Задание: Сочини короткий рассказ, обязательно упоминая следующие объекты: Таня, мячик, речка
Таня любила играть в волейбол. Она была хорошо известна по всему городу благодаря своим спортивным достижениям и таланту. Однажды, когда она отдыхала после тренировки на берегу реки, она заметила, что там много маленьких рыбашек. Она решила сделать им компанию и привезла с собой мячик. Рыбки были очень радостно удивлены, и они стали играть с ней. Таня бросала мяч в воду, а рыбки прыгали, пытаясь его поймать. Это было действительно весело и интересно. Вскоре другие люди, проходящие мимо, также присоединились к игре. Играя с рыбками, Таня забыла 

In [ ]:
# Загрузка страниц из википедии
!pip install llama-index-readers-wikipedia wikipedia
from llama_index.readers.wikipedia import WikipediaReader

# Инициализация объекта WikipediaReader
reader = WikipediaReader()

# Загрузка данных из википедии
docs = reader.load_data(
    pages=["Искусственный интеллект"],  # запрос раздела на тему ИИ
    lang_prefix = 'ru'                  # из рускозычной зоны википедии
    )

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=b34d5673278be48020d773c0d887b1c50b956761d49f099a90a9261465dc9a96
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
from langchain_huggingface  import HuggingFaceEmbeddings
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Настройка ServiceContext (глобальная настройка параметров LLM)
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:
# Создаем простое графовое хранилище
graph_store = SimpleGraphStore()

# Устанавливаем информацию о хранилище в StorageContext
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [ ]:
# Запускаем генерацию индексов из документа с помощью KnowlegeGraphIndex
indexKG = KnowledgeGraphIndex.from_documents( documents=docs,               # данные для построения графов
                                           max_triplets_per_chunk=3,        # сколько обработывать триплетов связей для каждого блока данных
                                           show_progress=True,              # показывать процесс выполнения
                                           include_embeddings=True,         # включение векторных вложений в индекс для расширенной аналитики
                                           storage_context=storage_context) # куда сохранять результаты

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Processing nodes:   0%|          | 0/96 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/13 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/32 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/112 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/14 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
from pyvis.network import Network
from IPython.display import display
import IPython

g = indexKG.get_networkx_graph(500)
net = Network(notebook=True,cdn_resources="in_line", directed=True)
net.from_nx(g)
net.show("graph.html")
net.save_graph("Knowledge_graph.html")

IPython.display.HTML(filename="/content/Knowledge_graph.html")

graph.html


In [ ]:
query = "Расскажи о финансовых роботах консультантах. Как они могут улучшить мое финансовое состояние?"
query_engine = indexKG.as_query_engine(include_text=True, verbose=True)
#
message_template =f"""<s>system
Отвечай в соответствии с Источником. Проверь, есть ли в Источнике упоминания о ключевых словах Вопроса.
Если нет, то просто скажи: 'я не знаю'. Не придумывай! </s>
<s>user
Вопрос: {query}
Источник:
</s>
"""
#
response = query_engine.query(message_template)
#
print()
print('Ответ:')
print(response.response)


Extracted keywords: ['улучшение', 'Консультанты', 'финансовые', 'состояние', 'финансовое']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Automated financial advisor', 'Works on', 'Algorithm')
('Wallet.ai', 'Informs', 'Users about their expenses')

Ответ:
Конечно, я могу рассказать вам о финансовых роботах-консультантах. Они являются автоматизированными помощниками, которые предоставляют финансовые консультации и дают советы по управлению финансовым портфелем. Они работают на основе алгоритмов, которые были разработаны для автоматического развития портфеля в соответствии с инвестиційными целями и склонностью к риску крупного инвестора. Они также могут корректировать изменения на рынке и калибраживать портфель в соответствии с желаниями клиента.


In [ ]:
storage_context.persist() # сохраняем хранилище в файловую систему

После подготовки данных приступаем к выполнинию ДЗ.

In [ ]:
!pip install gradio transformers peft torch llama-index-readers-file sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.2
    Uninstalling tomlkit-0.13.2:
      Successfully uninstalled tomlkit-0.13.2


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import PeftModel, PeftConfig
from transformers import BitsAndBytesConfig

# Задаем имя модели
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"

# Определяем параметры квантования
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Создание конфига для PEFT
config = PeftConfig.from_pretrained(MODEL_NAME)

# Загружаем базовую модель
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=quantization_config,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Загружаем LoRA модель
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)

# Переводим модель в режим инференса
model.eval()

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM

# Инициализация LLM
llm = HuggingFaceLLM(
    model=model,
    model_name=MODEL_NAME,
    tokenizer=tokenizer,
    max_new_tokens=generation_config.max_new_tokens,
    model_kwargs={"quantization_config": quantization_config},
    generate_kwargs={
        "bos_token_id": generation_config.bos_token_id,
        "eos_token_id": generation_config.eos_token_id,
        "pad_token_id": generation_config.pad_token_id,
        "no_repeat_ngram_size": generation_config.no_repeat_ngram_size,
        "repetition_penalty": generation_config.repetition_penalty,
        "temperature": generation_config.temperature,
        "do_sample": True,
        "top_k": 50,
        "top_p": 0.95
    },
    device_map="auto",
)


In [ ]:
from llama_index.readers.file import PDFReader
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.core import KnowledgeGraphIndex, Settings, StorageContext
from llama_index.core.graph_stores import SimpleGraphStore

# Функция для парсинга данных из PDF
def parse_pdf_data(pdf_file):
    reader = PDFReader()
    docs = reader.load_data(file=pdf_file.name)

    embed_model = LangchainEmbedding(
        HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    )

    Settings.llm = llm
    Settings.embed_model = embed_model
    Settings.chunk_size = 512

    graph_store = SimpleGraphStore()
    storage_context = StorageContext.from_defaults(graph_store=graph_store)

    indexKG = KnowledgeGraphIndex.from_documents(
        documents=docs,
        max_triplets_per_chunk=3,
        show_progress=True,
        include_embeddings=True,
        storage_context=storage_context
    )

    return indexKG

# Функция для обработки запроса
def process_query(pdf_file, query):
    indexKG = parse_pdf_data(pdf_file)
    query_engine = indexKG.as_query_engine(include_text=True, verbose=True)

    message_template =f"""<s>system
Отвечай в соответствии с Источником. Проверь, есть ли в Источнике упоминания о ключевых словах Вопроса.
Если нет, то просто скажи: 'я не знаю'. Не придумывай! </s>
<s>user
Вопрос: {query}
Источник:
</s>
"""

    response = query_engine.query(message_template)
    return response.response


In [ ]:
import gradio as gr

# Определяем интерфейс Gradio
with gr.Blocks() as demo:
    gr.Markdown("## Модель для работы с графом знаний на основе данных из PDF файла")

    with gr.Row():
        with gr.Column():
            pdf_input = gr.File(label="Загрузите PDF файл")
            query_input = gr.Textbox(label="Введите ваш вопрос")
            submit_button = gr.Button("Распарсить данные и отправить запрос")

        with gr.Column():
            response_output = gr.Textbox(label="Ответ от модели")

    submit_button.click(fn=process_query, inputs=[pdf_input, query_input], outputs=response_output)

# Запуск интерфейса
demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0b2075fe62016c4915.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
